# DashDB to Object Storage Load
This is a notebook to archive the data stored in DashDB and save it in the Parquet File Format in the Bluemix Object Storage.

In [41]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Add Data Source
go to the Data Source tab on the right hand menu,  add source, from bluemix, and select my-dashdb. Next, select the line below and click on "Insert to code". The name of the credentials object should be **credentials_1**. Then click on the menu item "Cell" and click on "Run All".

In [43]:
from IPython.display import HTML
HTML('''<style>
div#notebook {
    background-color: #fafbfc;
    border-top: none;
}

div#notebook-container {
    background-color: #fafbfc;
}
</style>
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [44]:
url = credentials_1["jdbcurl"]+":user="+credentials_1["username"]+";password="+credentials_1["password"]+";"
table = credentials_1["username"]+".MYTABLE"
print "URL:", url
print "Table:", table

URL: jdbc:db2://dashdb-entry-yp-dal09-09.services.dal.bluemix.net:50000/BLUDB:user=dash6233;password=TNthBxudaJtf;
Table: dash6233.MYTABLE


In [45]:
# save dashDB data to DataFrame
df = sqlContext.read.format('jdbc').options(url=url, dbtable=table).load()

In [46]:
# register data with table name
df.registerTempTable("simdata")

In [47]:
# query the table and save results to DataFrame
df = sqlContext.sql("SELECT * FROM simdata")

First five elements

In [48]:
df.toPandas().head()

,TS,TEMPERATURE,LIGHT,DOOROPEN,WINDOWOPEN,HUMIDITY,OXYGEN,WATERCONSUMPTION,ENERGYCONSUMPTION
0,2016-06-01T12:08:25.279Z,20,482,0,0,49,20,9,10
1,2016-06-01T12:09:25.283Z,20,481,0,0,45,20,10,10
2,2016-06-01T12:10:25.284Z,18,496,0,0,49,21,10,9
3,2016-06-01T12:15:25.294Z,18,498,0,0,51,20,10,10
4,2016-06-01T12:16:25.302Z,21,502,0,0,45,20,9,9


Last 5 elements

In [49]:
df.toPandas().tail()

,TS,TEMPERATURE,LIGHT,DOOROPEN,WINDOWOPEN,HUMIDITY,OXYGEN,WATERCONSUMPTION,ENERGYCONSUMPTION
8129,2016-06-15T09:13:33.133Z,20,515,0,0,48,20,10,9
8130,2016-06-15T09:14:33.137Z,18,507,0,0,49,21,9,9
8131,2016-06-15T09:15:33.143Z,21,489,0,0,54,20,10,9
8132,2016-06-15T09:16:33.147Z,20,502,0,0,46,21,10,9
8133,2016-06-15T09:17:33.153Z,19,493,0,0,52,21,10,9


In [50]:
print "Count:", df.count()

Count: 8134


In [51]:
# you can filter data you want to get here
tosave = sqlContext.sql("SELECT * FROM simdata")

save as parquet file into object storage

In [52]:
# Save as parquet file
tosave.write.parquet("swift://my-container.spark/simdata.parquet", mode="overwrite")

read from object storage to verify success

In [53]:
parquetFile = sqlContext.read.parquet("swift://my-container.spark/simdata.parquet")
parquetFile.registerTempTable("saved")

In [54]:
savedfile = sqlContext.sql("SELECT * FROM saved")

In [55]:
savedfile.toPandas().head()

,TS,TEMPERATURE,LIGHT,DOOROPEN,WINDOWOPEN,HUMIDITY,OXYGEN,WATERCONSUMPTION,ENERGYCONSUMPTION
0,2016-05-31T10:21:11.855Z,10,50,0,0,7,45,87,23
1,2016-05-31T10:22:11.868Z,19,514,0,0,51,20,10,10
2,2016-05-31T10:23:11.884Z,18,486,0,0,45,21,9,10
3,2016-05-31T10:24:11.893Z,20,485,0,0,45,21,9,10
4,2016-05-31T10:25:11.909Z,18,505,0,0,45,20,9,10


In [56]:
savedfile.count()

8134